In [2]:
import os
import numpy as np
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image
from torch.utils.data import random_split

import torchattacks

from tqdm import tqdm
import math
import random
import csv
import torchvision.transforms as transforms
from PIL import Image

## Load adversarial examples from different sources

In [116]:
if False:
    def recreate_folder(folder_path):
        if os.path.exists(folder_path):
            os.system(f"rm -rf {folder_path}")
        os.system(f"mkdir {folder_path}")

    recreate_folder("all_adv_images")
    recreate_folder("all_FGSM_adv_images")
    recreate_folder("all_MIFGSM_adv_images")
    recreate_folder("all_PGD_adv_images")
    cnt = 0
    for folders in os.listdir():
        if folders.split("_")[0] == "resnet110":
            os.chdir(f"{folders}/adv_imgs")
            attack_type = folders.split("_")[-1]
            cnt += 1
            for images in os.listdir():
                os.system(f"cp {images} ../../all_adv_images/{images.split(".")[0]}_{cnt}_{attack_type}.png")
                os.system(f"cp {images} ../../all_{attack_type}_adv_images/{images.split(".")[0]}_{cnt}_{attack_type}.png")
            os.chdir("../../")

## Start from here !!

In [180]:
config = {
    "train_batch_size": 64,
    "batch_size": 1,
    "model_name": "resnet110_cifar100",
    "data_filepath": "./cifar-100_eval",
    "train_adv_imgs_filepath": "./all_PGD_adv_images",
    "test_adv_imgs_filepath": "./all_MIFGSM_adv_images",
    "epoch": 2,
    "seeds": 10901036,
    "learning_rate": 0.001,
}
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [181]:
from pytorchcv.model_provider import get_model as ptcv_get_model
model = ptcv_get_model(config["model_name"], pretrained=True).to(device)

## Train the model with adversarial images (adversarial training)

In [182]:
class Cifar_Adveresarial100(Dataset):
    def __init__(self, data, labels):
        super(Cifar_Adveresarial100, self).__init__()
        self.data = data
        self.labels = labels
        self.datasize = len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
            
    def __len__(self):
        return self.datasize

In [183]:
original_files = sorted([os.path.join(config["data_filepath"],x) for x in os.listdir(config["data_filepath"]) if x.endswith(".png")])
train_adv_files = sorted([os.path.join(config["train_adv_imgs_filepath"], x) for x in os.listdir(config["train_adv_imgs_filepath"]) if x.endswith(".png")])
test_adv_files = None

train_size = int(0.7 * len(train_adv_files))
val_size = int(0.15 * len(train_adv_files))
test_size = len(train_adv_files) - train_size - val_size
original_size = len(original_files)
train_files, val_files, test_files = random_split(train_adv_files, [train_size, val_size, test_size])

train_imgs = torch.stack([transforms.ToTensor()(Image.open(filename)) for filename in train_files])
val_imgs = torch.stack([transforms.ToTensor()(Image.open(filename)) for filename in val_files])
test_imgs = torch.stack([transforms.ToTensor()(Image.open(filename)) for filename in test_files])
original_imgs = torch.stack([transforms.ToTensor()(Image.open(filename)) for filename in original_files])

train_labels = torch.tensor([int(filename.split('/')[-1].split('_')[0]) for filename in train_files], dtype=torch.long)
val_labels = torch.tensor([int(filename.split('/')[-1].split('_')[0]) for filename in val_files], dtype=torch.long)
test_labels = torch.tensor([int(filename.split('/')[-1].split('_')[0]) for filename in test_files], dtype=torch.long)
original_labels = torch.tensor([int(filename.split('/')[-1].split('_')[0]) for filename in original_files], dtype=torch.long)

train_datasets  = Cifar_Adveresarial100(train_imgs, train_labels)
val_datasets    = Cifar_Adveresarial100(val_imgs, val_labels)
test_datasets   = Cifar_Adveresarial100(test_imgs, test_labels)
original_datasets = Cifar_Adveresarial100(original_imgs, original_labels)

train_loader    = DataLoader(train_datasets, batch_size=config['train_batch_size'], shuffle=True, pin_memory=True)
val_loader      = DataLoader(val_datasets , batch_size=config['batch_size'], shuffle=False, pin_memory=True)
test_loader     = DataLoader(test_datasets, batch_size=config['batch_size'], shuffle=False, pin_memory=True)
original_loader = DataLoader(original_datasets, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

In [184]:
another_test_adv_files = sorted([os.path.join(config["test_adv_imgs_filepath"], x) for x in os.listdir(config["test_adv_imgs_filepath"]) if x.endswith(".png")])
another_test_imgs = torch.stack([transforms.ToTensor()(Image.open(filename)) for filename in another_test_adv_files])
another_test_labels = torch.tensor([int(filename.split('/')[-1].split('_')[0]) for filename in another_test_adv_files], dtype=torch.long)
another_test_datasets   = Cifar_Adveresarial100(another_test_imgs, another_test_labels)
another_test_loader     = DataLoader(test_datasets, batch_size=config['batch_size'], shuffle=False, pin_memory=True)


In [185]:
print(f"Original        size: {len(original_datasets)}")
print(f"Training        size: {len(train_datasets)}")
print(f"Validating      size: {len(val_datasets)}")
print(f"Testing         size: {len(test_datasets)}")
print(f"Another Testing size: {len(another_test_datasets)}")

Original        size: 500
Training        size: 1400
Validating      size: 300
Testing         size: 300
Another Testing size: 2000


## Start training

In [186]:
criteria = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config["learning_rate"])
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

def train(model, train_loader, val_loader, criteria, optimizer, config):
    for epoch in range(config['epoch']):
        model.train()
        for idx, (data, label) in tqdm(enumerate(train_loader)):
            data, label = data.to(device), label.to(device)
            output = model(data)
            loss = criteria(output, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if idx % 5 == 0:
                print(f"Epoch: {epoch}, Iter: {idx}, Loss: {loss.item()}")
        
        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for idx, (data, label) in tqdm(enumerate(val_loader)):
                data, label = data.to(device), label.to(device)
                output = model(data)
                loss = criteria(output, label)
                _, predicted = torch.max(output, 1)
                total += label.size(0)
                correct += (predicted == label).sum().item()
            print(f"Epoch: {epoch}, Val Acc: {correct / total}, Val Loss: {loss.item()}")
        scheduler.step()

In [187]:
def predict(test_loader, model, device, data_size):
    model.eval() # Set your model to evaluation mode.
    acc = 0
    for img, label in tqdm(test_loader):
        img = img.to(device)
        label = label.to(device)
        with torch.no_grad():                   
            pred = model(img) 
            if label[0] == torch.argmax(pred):
                acc += 1
    return acc / data_size



In [188]:
original_acc_before = predict(original_loader, model, device, original_size)
attack_acc_before = predict(test_loader, model, device, test_size)
another_test_acc_before = attack_acc_before
if config["test_adv_imgs_filepath"] != config["train_adv_imgs_filepath"]:
    another_test_acc_before = predict(another_test_loader, model, device, len(another_test_datasets))

print(f"Adversarially trained on: {config["train_adv_imgs_filepath"]}\n")
print(f"Tested on: {config["train_adv_imgs_filepath"]} and {config["test_adv_imgs_filepath"]}\n")
    

print(f"Before adversarial training:\n\tOriginal_accuracy\t: {original_acc_before}\n\tAdv_accuracy\t\t: {attack_acc_before}\n\tAnother_test_accuracy\t: {another_test_acc_before}")
print("")

train(model, train_loader, val_loader, criteria, optimizer, config)

defense_acc_after = predict(test_loader, model, device, test_size)
original_acc_after = predict(original_loader, model, device, original_size)
another_test_acc_after = predict(another_test_loader, model, device, len(another_test_datasets))

print(f"After adversarial training:\n\tOriginal_accuracy\t: {original_acc_after}\n\tAdv_accuracy\t: {defense_acc_after}\n\tAnother_test_accuracy\t: {another_test_acc_after}")
print("")

with open("defense_info.txt", 'a') as f:
    f.write(f"Model: {config["model_name"]}\n")
    f.write(f"Adversarially trained on: {config["train_adv_imgs_filepath"]} with epochs = {config["epoch"]}\n")
    f.write(f"Tested on: {config["train_adv_imgs_filepath"]} and {config["test_adv_imgs_filepath"]}\n")
    f.write(f"Before adversarial training:\n\tOriginal_accuracy\t: {original_acc_before}\n\tAdv_accuracy\t\t: {attack_acc_before}\n\tAnother_test_accuracy\t: {another_test_acc_before}\n")
    f.write(f"After adversarial training:\n\tOriginal_accuracy\t: {original_acc_after}\n\tAdv_accuracy\t\t: {defense_acc_after}\n\tAnother_test_accuracy\t: {another_test_acc_after}\n")
    f.write("\n---------------------------------------------\n\n")

100%|██████████| 300/300 [00:03<00:00, 88.36it/s]


Adversarially trained on: ./all_PGD_adv_images

Tested on: ./all_PGD_adv_images and ./all_MIFGSM_adv_images

Before adversarial training:
	Original_accuracy	: 0.808
	Adv_accuracy		: 0.07333333333333333
	Another_test_accuracy	: 0.011



1it [00:01,  1.01s/it]

Epoch: 0, Iter: 0, Loss: 1.731886625289917


6it [00:06,  1.01s/it]

...

Epoch: 1, Iter: 20, Loss: 0.027064047753810883


22it [00:21,  1.00it/s]
300it [00:03, 86.70it/s]


Epoch: 1, Val Acc: 0.9833333333333333, Val Loss: 0.0035271355882287025


100%|██████████| 300/300 [00:03<00:00, 86.83it/s]

After adversarial training:
	Original_accuracy	: 0.994
	Adv_accuracy	: 0.9766666666666667
	Another_test_accuracy	: 0.1465

